In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn; seaborn.set()
from sklearn.decomposition import PCA


## Read input data

In [ ]:
casualty_df = pd.read_csv("dft-road-casualty-statistics-casualty-1979-2020.csv")

In [ ]:
accident_df = pd.read_csv("dft-road-casualty-statistics-accident-1979-2020.csv")

In [ ]:
vehicles_df = pd.read_csv("dft-road-casualty-statistics-vehicle-1979-2020.csv")

In [ ]:
merged_left = pd.merge(left=accident_df, right=vehicles_df, how='left', left_on='accident_index', right_on='accident_index')

In [ ]:
merged_left = pd.merge(left=merged_left, right=casualty_df, how='left', left_on='accident_index', right_on='accident_index')

In [ ]:
merged_left.shape

In [ ]:
merged_left.to_csv('all_uk_1979_2020_road_accidents.csv')

In [ ]:
merged_left = pd.read_csv("all_uk_1979_2020_road_accidents.csv")
merged_left = merged_left.drop('Unnamed: 0', axis = 1)

In [ ]:
all_data_post_2020 = merged_left.loc[merged_left['accident_year_x'] > 1999]

In [ ]:
no_loc_df_post_2020 = all_data_post_2020.drop(["accident_reference_x", "location_easting_osgr", "location_northing_osgr", "longitude", "latitude"], axis=1)

In [ ]:
no_loc_df_post_2020.head()

In [ ]:
no_loc_df_post_2020.shape

In [ ]:
no_loc_df_post_2020.sample(n=1000, random_state=1)


In [ ]:
clean_pca = PCA(n_components=8)
principalComponents = clean_pca.fit_transform(no_loc_df_post_2020)
principalDf = pd.DataFrame(data = principalComponents
             )
principalDf

In [ ]:
pd.Series(clean_pca.explained_variance_ratio_).plot(logy=True)